In [1]:
import os
import glob
import torch
import torch.nn as nn

from tqdm import tqdm
from torch.optim import SGD
from sklearn.datasets import make_moons
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

In [2]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.coef = nn.Parameter(torch.randn((1, 2)))
        self.bias   = nn.Parameter(torch.randn(1))
    
    def forward(self, x):
        return 1 / (1 + torch.exp(-(self.bias + torch.sum(self.coef * x, dim = 1, keepdims = True))))

In [3]:
class Dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx].unsqueeze(dim = -1).to(torch.float64)

In [4]:
class Saver:
    def __init__(self, model_name, model_dir):
        self.model_base_name = model_name
        self.model_dir = model_dir

        os.makedirs(self.model_dir, exist_ok = True)

    def start_or_resume_training(self, model):
        models_paths = sorted(glob.glob(os.path.join(self.model_dir, f'{self.model_base_name}_*.pth')))
        if len(models_paths) == 0:
            return 1, model
        last_model_path = models_paths[-1]
        last_model_epoch = int(os.path.splitext(last_model_path)[0].split('_')[-1])
        print(f'Resuming training from epoch {epoch}...')
        model.load_state_dict(torch.load(last_model_path))
        return last_model_epoch, model

    def save_model(self, model, epoch = None):
        model_name = self.model_base_name
        if epoch is not None:
            model_name += f'_{epoch}'
        
        previous_models_paths = glob.glob(os.path.join(self.model_dir, f'{self.model_base_name}_*.pth'))
        for previous_model_path in previous_models_paths:
            os.remove(previous_model_path)

        torch.save(model.state_dict(), os.path.join(self.model_dir, model_name + '.pth'))
        

In [5]:
X, y = make_moons(n_samples = 10000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

In [6]:
train_dataset = Dataset(X_train, y_train)
test_dataset  = Dataset(X_test,  y_test)
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader  = DataLoader(test_dataset,  batch_size = 1, shuffle = False)

In [7]:
model = LogisticRegression()
optimizer = SGD(model.parameters())
criterion = nn.BCELoss()
writer = SummaryWriter()
saver = Saver(model_name = 'model', model_dir = 'models')
graph_logged = False

In [8]:
n_epochs = 250
save_interval = 10
initial_epoch, model = saver.start_or_resume_training(model)

model.train()
for epoch in range(initial_epoch, n_epochs + 1):
    print(f'Epoch {epoch}/{n_epochs}')

    total_loss = 0
    for i, (x, y_true) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        y_pred = model(x)
        if not graph_logged:
            writer.add_graph(model, x)
            graph_logged = True
        loss = criterion(y_pred, y_true)
        loss_value = loss.item()
        step = (epoch - 1) * len(train_dataloader) + i
        writer.add_scalar("Loss/batch_train_loss", loss_value, step)
        total_loss += loss_value
        loss.backward()
        optimizer.step()
    
    epoch_loss = total_loss / len(train_dataloader)
    writer.add_scalar("Loss/epoch_train_loss", epoch_loss, epoch)
    print('Training loss:', epoch_loss)

    if epoch % save_interval == 0:
        saver.save_model(model, epoch)

saver.save_model(model)



Epoch 1/250


  0%|          | 0/250 [00:00<?, ?it/s]

100%|██████████| 250/250 [00:00<00:00, 269.37it/s]


Training loss: 1.352207160867811
Epoch 2/250


100%|██████████| 250/250 [00:00<00:00, 467.06it/s]


...

Training loss: 0.268987564023394
Epoch 249/250


100%|██████████| 250/250 [00:00<00:00, 474.69it/s]


Training loss: 0.26883448295922524
Epoch 250/250


100%|██████████| 250/250 [00:00<00:00, 477.72it/s]

Training loss: 0.2686825186070905


In [13]:
model.eval()

with torch.no_grad():
    total_loss = 0
    correct = 0
    for i, (x, y_true) in enumerate(tqdm(test_dataloader)):
        y_pred = model(x)
        if y_true == torch.round(y_pred):
            correct += 1
        loss = criterion(y_pred, y_true)
        total_loss += loss.item()
    
    test_loss = total_loss / len(test_dataloader)
    writer.add_scalar("Loss/test_loss-X", test_loss)
    print('Test loss:', test_loss)
    print('Accuracy:', correct / len(test_dataloader)) # TODO: substituir por acurácia balanceada

100%|██████████| 2000/2000 [00:00<00:00, 2038.57it/s]

Test loss: 0.25924823939506075
Accuracy: 0.877


In [10]:
writer.close()